In [344]:
%%opts Scatter [width=800, height=400]
x = np.linspace(0,15,1000)
y = np.fabs(np.sin(x)*.1)+np.random.rand(len(x))*.1
data =np.vstack([x,y]).T
hv.Scatter(data=data)

:Scatter   [x]   (y)

In [106]:
data

array([[ 0.        ,  0.03153052],
       [ 0.01501502,  0.05946089],
       [ 0.03003003,  0.12832236],
       ...,
       [14.96996997,  0.69066788],
       [14.98498498,  0.71119544],
       [15.        ,  0.66253762]])

In [92]:
from scipy.signal import savgol_filter
from sklearn.linear_model import LinearRegression

In [47]:
model = LinearRegression()

In [342]:
def bounce_detection(x,y, steps=10, threshold=0):
    STEPS = steps
    THRESHOLD = threshold
    x = np.flip(x, axis=0).reshape(-1,1)
    y = np.flip(y, axis=0).reshape(-1,1)
#    x = x.reshape(-1,1)
#    y = y.reshape(-1,1)
    smoothed = y.T#savgol_filter(y.T, 11, 2)
    diffx = [x[i,0]-x[i-1,0] for i in range(1,len(x))]
    diffy = [smoothed[0][i]-smoothed[0][i-1] for i in range(1,len(smoothed[0]))]

    jump = 0
    lr = [[x[0][0], smoothed[0][0]]]
    for i in range(STEPS,len(diffy)-STEPS):
        if jump > 0:
            jump -= 1
            continue

        d_v = sum(diffy[i-STEPS:i])
        d_n = sum(diffy[i: i+STEPS])

        left_ = True
        right_ = True

        bound = [x[i,0]-lr[-1][0], smoothed[0,i]-lr[-1][1]]
        
        if x[0] < x[i]:
            for j in range(STEPS):
                if np.cross(bound, [x[i-j,0]-x[i,0], smoothed[0,i-j]-smoothed[0,i]]) < THRESHOLD:
                    left_ = False
                if np.cross(bound, [x[i+j,0]-x[i,0], smoothed[0,i+j]-smoothed[0,i]]) < THRESHOLD:
                    right_ = False
                    
            if d_v<0 and d_n>-0.5*d_v and left_ and right_:
                lr.append([x[i][0], y[i][0]])
                jump = STEPS
        else:
            for j in range(STEPS):
                if np.cross(bound, [x[i-j,0]-x[i,0], smoothed[0,i-j]-smoothed[0,i]]) > THRESHOLD:
                    left_ = False
                if np.cross(bound, [x[i+j,0]-x[i,0], smoothed[0,i+j]-smoothed[0,i]]) > THRESHOLD:
                    right_ = False
            if right_ and left_:
                print('yesyes', x[i], d_n, d_v)
            if d_v<0 and d_n>-0.5*d_v and left_ and right_:
                lr.append([x[i][0], y[i][0]])
                jump = STEPS
    return lr

In [340]:
x[-163]

12.567567567567567

In [343]:
%%output size=150
%%opts Scatter [width=200, height=100]
lr = bounce_detection(x=x, y=y, threshold=0)
hv.Scatter(data=data)*hv.Scatter(data=lr)#*hv.Scatter(crs)#*hv.Scatter(np.vstack([x[1:].T, diffy]).T)

yesyes [14.66966967] 0.14255836334802208 0.08951407632151276
yesyes [12.5975976] 0.17825555646385424 -0.2446888043104646
yesyes [9.41441441] 0.12272494874166082 -0.12377246753909874
yesyes [6.27627628] 0.17939108867462109 -0.11773548825847435
yesyes [5.93093093] 0.17345156541377055 0.05218100348302712
yesyes [5.75075075] 0.19285600805871195 0.08622820560078387
yesyes [5.46546547] 0.17618679540853743 0.03565935781921192
yesyes [3.13813814] 0.18437647934328913 -0.14330456848072143
yesyes [2.46246246] 0.20767722761769525 0.1137030800951625


:Overlay
   .Scatter.I  :Scatter   [x]   (y)
   .Scatter.II :Scatter   [x]   (y)

In [328]:
s1 = np.array([15,10])
s2 = np.array([13,.1])
s3 = np.array([12.9,.2])
np.cross(s2-s1, s3-s2)

array(-1.19)

In [169]:
%%output size=150
%%opts Scatter [width=800, height=400]
%%opts Curve [width=800, height=400]

smoothed = savgol_filter(y.T, 11, 2)

hv.Curve(data=data)*hv.Curve(data=np.vstack([x.T,smoothed]).T)

:Overlay
   .Curve.I  :Curve   [x]   (y)
   .Curve.II :Curve   [x]   (y)

In [134]:
len(diffy)

999

In [112]:
len(y)

1000

In [175]:
x[0], diffy[0]

(array([0.]), 0.02144281271707746)

In [191]:
crs=[]
for i in range(len(diffy)-1):
    crs.append([x[i,0],np.cross([x[i,0]-x[1,0], smoothed[0,i]-smoothed[0,0]], [x[i+1,0]-x[i,0], smoothed[0,i+1]-smoothed[0,i]])])

In [192]:
hv.Scatter(crs)

:Scatter   [x]   (y)

In [180]:
diffy

[0.02144281271707746,
 0.020134765220642142,
 0.018826717724206812,
 0.017518670227771496,
 0.01621062273133661,
 0.013239118067021494,
 0.007740836235647142,
 0.010209591165297738,
 0.016582857665811357,
 0.005837946899991064,
 0.013728170816460827,
 0.0013617798358798727,
 0.01654530531982129,
 0.0058589180165679045,
 0.02326774948548488,
 0.024914799982982294,
 0.018453589006722004,
 0.005563085489913855,
 0.0178412947065249,
 0.013340651959305005,
 0.016172706351092414,
 0.0141633536189964,
 0.02544238199585186,
 0.006409539081607907,
 0.024205932307889477,
 0.013236957681256989,
 0.012765983122851265,
 0.002706037409835216,
 0.005420006614782091,
 0.013735792928442792,
 0.003083388242700158,
 0.007128446709896652,
 0.017119110138405502,
 0.01790467310290489,
 0.022119255675280725,
 0.023051846243191232,
 0.02193796436862583,
 0.010791185814511994,
 0.012155663622773205,
 0.015313815173384815,
 0.014276399939175732,
 0.0018421131505361732,
 0.012171412966087924,
 0.0142781865697164